In [ ]:
from IPython.core.display import HTML
with open('./style.css') as f:
    css = f.read()
HTML(css)

In [ ]:
import chess
import chess.gaviota
from IPython.display import display

# Introduction

This notebook serves as an introduction to the student research project and definitions of key terms are introduced. To be able to follow the work, it is assumed to understand the game of chess as far as possible. It should be known what pieces exist, what moves they can make, when a player is checkmate, when a game ends in stalemate and any other basics needed to play a game of chess. This student research project introduces both the concept of Endgame Tablebase and Retrograde Analysis and how they are used to play an endgame "perfectly". Furthermore, an own program for playing chess endgames is developed, which handles different endgames using custom created tablebases. To simulate the chess game itself the Python library [chess](https://python-chess.readthedocs.io/en/latest/) is used.

<b><mark>Definition (Pythonic Chess Piece)</mark></b> [Click here for documentation](https://python-chess.readthedocs.io/en/latest/core.html#chess.Piece)

A <span style="color:blue">Pythonic Chess Piece</span> (also called Piece) is the Python implementation of the chess pieces that are used when playing a game of chess. For each color (white or black) there are 6 types of chess pieces: pawn, bishop, knight, rook, queen and king. Chess pieces are implemented as a separate class and can be created using their constructor: 

&nbsp;&nbsp;&nbsp;`chess.Piece(type, color)`

where 

1. $\text{type} \in \{\text{chess.PAWN}, \text{chess.KNIGHT}, \text{chess.BISHOP}, \text{chess.ROOK}, \text{chess.QUEEN}, \text{chess.KING} \} \text{ resp. } \{1,...,6\}$
2. $\text{color} \in \{\text{chess.WHITE}, \text{chess.BLACK}\} \text{ resp. } \{\text{True}, \text{False}\}$

The above values for type and color are only predefined constants for a numeric or boolean value.

In [ ]:
print(f"chess.PAWN = {chess.PAWN}, chess.KNIGHT = {chess.KNIGHT}, chess.BISHOP = {chess.BISHOP}, chess.ROOK = {chess.ROOK}, chess.QUEEN = {chess.QUEEN}, chess.KING = {chess.KING}, chess.WHITE = {chess.WHITE}, chess.BLACK = {chess.BLACK}")

In [ ]:
display(chess.Piece(chess.KING, chess.WHITE), chess.Piece(6, False))

<b><mark>Definition (Piece Symbol)</mark></b>

A <span style="color:blue">Piece Symbol</span> is the char representation of a chess piece:

&nbsp;&nbsp;&nbsp;$ \text{piece_symbol} \in \{\text{'K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p'}\}$

Capitalization stands for the color and the respective character for the piece type:


<table align="left">
  <tr>
    <th></th>
    <th>King</th>
    <th>Queen</th>
    <th>Rook</th>
    <th>Bishop</th>
    <th>Knight</th>
    <th>Pawn</th>
  </tr>
  <tr>
    <th>White</th>
    <td style="text-align: center;">K</td>
    <td style="text-align: center;">Q</td>
    <td style="text-align: center;">R</td>
    <td style="text-align: center;">B</td>
    <td style="text-align: center;">N</td>
    <td style="text-align: center;">P</td>
  </tr>
  <tr>
    <th>Black</th>
    <td style="text-align: center;">k</td>
    <td style="text-align: center;">q</td>
    <td style="text-align: center;">r</td>
    <td style="text-align: center;">b</td>
    <td style="text-align: center;">n</td>
    <td style="text-align: center;">p</td>
  </tr>
</table>

### Equivalence of Pythonic Chess Pieces and Piece Symbols

Pythonic Chess Pieces and Piece Symbols are equivalent representations of chess pieces. They can be freely converted into each other:

<table align="left">
  <tr>
    <th><code>chess.Piece.from_symbol({piece_symbol})</code></th>
    <td>piece_symbol</td>
    <td>►</td>
    <td>pythonic_chess_piece</td>
  </tr>
  <tr>
    <th style="text-align: left;"><code>{pythonic_chess_piece}.symbol()</code></th>
    <td>pythonic_chess_piece</td>
    <td>►</td>
    <td style="text-align: left;">piece_symbol</td>
  </tr>
</table>

In [ ]:
pythonic_chess_piece = chess.Piece(chess.KING, chess.WHITE)
piece_symbol = 'K'

print(chess.Piece.from_symbol(piece_symbol) == pythonic_chess_piece)
print(pythonic_chess_piece.symbol() == piece_symbol)

<b><mark>Definition (Pythonic Chess Board)</mark></b> [Click here for documentation](https://python-chess.readthedocs.io/en/latest/core.html#board)

A <span style="color:blue">Pythonic Chess Board</span> (also called Board) is the Python implementation of a chess board. A Pythonic Chess Board is implemented as a separate class and can be created via the corresponding constructor:

&nbsp;&nbsp;&nbsp;`chess.Board({FEN})`

where

- FEN is an optional argument to create a concrete board (see Definition of FEN String)

In [ ]:
board = chess.Board()
board

A Board is called <span style="color:blue">valid</span> iff specific status checks are met ([Click here for documentation](https://python-chess.readthedocs.io/en/latest/core.html?highlight=is_legal#chess.Board.status)). This does not imply that the position is actually reachable with a series of legal moves from the starting position. To check all validation requirements at once the method `is_valid()` can be used:

In [ ]:
board.is_valid()

A Board <span style="color:blue">is_checkmate</span> iff `{pythonic_chess_board}.is_checkmate() = True`. This is the standard implementation of the chess rules.

A Board <span style="color:blue">is_stalemate</span> iff `{pythonic_chess_board}.is_stalemate() = True`. This is the standard implementation of the chess rules.

The set of all Pythonic Chess Boards is called <span style="color:blue">BOARD</span>.

<b><mark>Definition (FEN String)</mark></b>

Steven Edwards, creator of the FEN Standard, describes FEN as follows:

> FEN is "Forsyth-Edwards Notation"; it is a standard for describing chess
positions using the ASCII character set. A single FEN record uses one text line of variable length composed of six data
fields. The first four fields of the FEN specification are the same as the
first four fields of the EPD specification. [[Standard: Portable Game Notation Specification and Implementation Guide](https://www.thechessdrum.net/PGN_Reference.txt) 16.1: FEN by Steven Edwards]

A FEN string uses piece symbols and consists of several components:
- Piece Placement
- Side to move
- Castling ability
- En passant target square
- Halfmove clock
- Fullmove counter

The set of all FEN strings is written as <span style="color:blue">FEN</span>. The exact structure of these individual components can be found in the standard cited above. In the following is an example of how a concrete board can be created with a FEN string:

In [ ]:
chess.Board('8/8/1k6/2p5/8/6R1/8/7K b KQkq - 1 2')

### Equivalence of Pythonic Chess Boards and FEN Strings

Pythonic Chess Boards and FEN strings are equivalent representations of chess boards and they contain the same information. They can be freely converted into each other:

<table align="left">
  <tr>
    <th style="text-align: left;"><code>chess.Board({fen_string})</code></th>
    <td>fen_string</td>
    <td>►</td>
    <td>pythonic_chess_board</td>
  </tr>
  <tr>
    <th><code>{pythonic_chess_board}.fen()</code></th>
    <td>pythonic_chess_board</td>
    <td>►</td>
    <td style="text-align: left;">fen_string</td>
  </tr>
</table>

In [ ]:
pythonic_chess_board = chess.Board()
fen_string = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'

print(chess.Board(fen_string) == pythonic_chess_board)
print(pythonic_chess_board.fen() == fen_string)

<b><mark>Definition (EPD String)</mark></b>

EPD strings are just a simplified form of FEN strings. For example, they only need the Piece Placement and Side to Move. The set of all EPD strings is written as <span style="color:blue">EPD</span>. The exact definition can also be found in the above quoted standard under 16.2 EPD. In the following is an example of how a concrete board can be created with a EPD string:

In [ ]:
chess.Board('8/8/1k6/2p5/8/6R1/8/7K b')

### Contextual equivalence of Pythonic Chess Boards and EPD strings.

In the following we will explain why EPD strings and pythonic chess boards are considered contextually equivalent. This is the case, because in the concrete context of this work not all information of a pythonic chess board is relevant. For example, the information Castling ability, En passant target square, Halfmove clock and Fullmove counter are not used in the course of the work and thus in the consideration of the endgames. For this reason the content of relevant information of an EPD string is sufficient for this concrete context and in the course of the work mainly EPD strings are used to represent a board outside the Python space. Nevertheless, EPD strings can be transformed into Pythonic chess boards and vice versa. These are however no equivalent transformations, because the information content of pythonic chess boards is higher!

<table align="left">
  <tr>
    <th style="text-align: left;"><code>chess.Board({epd_string})</code></th>
    <td>epd_string</td>
    <td>►</td>
    <td>pythonic_chess_board</td>
  </tr>
  <tr>
    <th><code>{pythonic_chess_board}.epd()</code></th>
    <td>pythonic_chess_board</td>
    <td>►</td>
    <td style="text-align: left;">epd_string</td>
  </tr>
</table>

<b><mark>Definition (Piece String)</mark></b>

The string concatenation of piece_symbols is called <span style="color:blue">piece_string</span> iff
1. $|\text{piece_string}| < 5$
2. $\text{'K'} \in \text{piece_string}$
3. $\text{'k'} \in \text{piece_string}$
4. $\text{piece_string is sorted: }K > Q > R > N > B > P > k > q > r > n > b > p$

The set of all piece_strings is written as <span style="color:blue">PIECE_STRING</span>.

Examples:
- $\text{'Kkr'} \in \text{PIECE_STRING}$
- $\text{'Kkrn'} \in \text{PIECE_STRING}$
- $\text{'Kkrnb'} \notin \text{PIECE_STRING}$
- $\text{'QRkb'} \notin \text{PIECE_STRING}$
- $\text{'Kknr'} \notin \text{PIECE_STRING}$

<b><mark>Definition (Piece List)</mark></b>

The list concatenation of pythonic chess pieces is called <span style="color:blue">piece_list</span> iff the same 4 conditions as for piece strings are fulfilled.

The set of all piece_lists is written as <span style="color:blue">PIECE_LIST</span>.

Examples:
- $\text{[chess.Piece(chess.KING, chess.WHITE), chess.Piece(chess.KING, chess.BLACK), chess.Piece(chess.ROOK, chess.BLACK)]} \in \text{PIECE_LIST}$
- $\text{[chess.Piece(6, True), chess.Piece(6, False), chess.Piece(4, False)]} \in \text{PIECE_LIST}$
- $\text{[chess.Piece(chess.KING, chess.WHITE), chess.Piece(chess.QUEEN, chess.BLACK), chess.Piece(chess.ROOK, chess.BLACK)]} \notin \text{PIECE_LIST}$

<b><mark>Definition (Winning Color)</mark></b>

A <span style="color:blue">winning_color</span> is a boolean value, which represents a particular color in chess:

&nbsp;&nbsp;&nbsp;$\text{winning_color}\in \{\text{True}, \text{False}\} \text{ resp. } \{\text{chess.WHITE}, \text{chess.BLACK}\}$

The set of all winning colors is written as <span style="color:blue">WINNING_COLOR</span> (WINNING_COLOR $ = \mathcal{B}$).

<b><mark>Definition (Endgame)</mark></b>

A Piece String called endgame describes an <span style="color:blue">Endgame</span> iff

&nbsp;&nbsp;&nbsp;$\text{endgame} \in \text{{'KRk', 'KQk', 'KBBk', 'KNNk', 'KBNk', 'KQkr', 'Kkr', 'Kkq', 'Kkbb', 'Kknn', 'Kkbn', 'KRkq'}}$.

The above set of all Piece Strings describing an endgame is called <span style="color:blue">ENDGAME</span>.

A concrete Endgame is now the set of all chess boards, in which the pieces, which are present in the piece string, occur. Note that for an Endgame with 4 pieces (or more), the chess boards of the sub-Endgames are also present in the set. For example, in the endgame 'KBNk' the endgame 'KBk' and 'KNk' are also included.

<b><mark>Definition (Depth to Mate)</mark></b>

The <span style="color:blue">depth to mate</span> is an integer $dtm \in \mathbb{Z}$. The absolute value is the number of half-moves until forced mate of a specific board. The value is positive if the side to move is winning, otherwise it is negative. If the value is 0, it is either checkmate, stalemate or neither player can force a checkmate ([Click here for documentation](https://python-chess.readthedocs.io/en/latest/gaviota.html#chess.gaviota.PythonTablebase.probe_dtm)). With the help of this value, the perfect move can be played. The basic idea is that for every legal move that is possible, you look at the depth to mate of the resulting board and choose the move that leads to the best depth to mate for you. The problem is that to know the depth-to-mate value of a position, the entire rest of the game must be calculated. For this reason, we can calculate it only for positions where there are few pieces (3-5, so only for endgames).

<b><mark>Definition (Tablebase)</mark></b>

A <span style="color:blue">Tablebase</span> is a relation which is defined over an endgame and assigns a depth to mate to each chess board of the endgame.

<b><mark>Definition (Pythonic Tablebase)</mark></b>

A <span style="color:blue">Pythonic Tablebase</span> is the python implementation of a tablebase as a class. A Pythonic tablebase must implement the function

&nbsp;&nbsp;&nbsp;<b>probe_dtm: PYTHONIC_CHESS_BOARD $\rightarrow$ $\mathbb{Z}$<b>.

The function gets an board and returns the corresponding depth to mate. Furthermore a close method must be implemented, so that open files can be closed. The set of Pythonic Tablebases is called <span style="color:blue">PYTHONIC_TABLEBASE</span>.

### Gaviota Tablebases

A concrete example of a Pythonic Tablebase is the python module `chess.gaviota` which is based on the [Gaviota endgame tablebases](https://www.chessprogramming.org/Gaviota_Tablebases) developed by Miguel A. Ballicora. In general, Gaviota supports endgames with up to 5 pieces. To use the Gaviota tablebases, the tablebases must be downloaded (see /gaviota) and the respective tablebase must be opened. Using the method `tablebase.probe_dtm({board})` the corresponding dtm can now be displayed. The example below shows a dtm value of 11 for the board. 
Interpretation: Value is positive, so white (its whites turn) can force a checkmate in 11 half moves.

In [ ]:
board = chess.Board('7k/8/8/8/8/6Q1/8/7K w')

display(board)

#context manager indirectly runs the close method
with chess.gaviota.open_tablebase('gaviota') as tablebase: 
    print(f"dtm: {tablebase.probe_dtm(board)}")

###  Structure and objective of the student research project

The goal of the student research project is the generation of custom tablebases with the help of the Retrograde Analysis. First of all a program is implemented, which allows to play endgames using the Gaviota tablebases. In the following an implementation of the Retrograde Analysis is made, i.e. the computation of the DTMs of all sitatuations of an endgame. Finally, the program is extended so that the custom created tablebases can also be used for playing. The results are checked in the conclusion with tests.